Check Python version first. It should be at least 3.5

In [1]:
import sys
print(sys.version)

3.5.2 |Anaconda 4.1.1 (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston

boston = load_boston()
dir(boston)

X_data = np.array(boston.data)
y_data = np.array(boston.target)

n = X_data.shape[0]
num_feat = X_data.shape[1]

Add a bias column to X and transform y into

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_data)

# This command is rather complicated
# This is taken from the book Hands on Machine Learning 
# X_bias = np.c_[np.ones((n,1)), X_scaled]
X_bias = np.column_stack([np.ones((n,1)), X_scaled])

yy = y_data.reshape((-1,1))
print("Current shape", yy.shape)
print("Previous shape", y_data.shape)

Current shape (506, 1)
Preiovus shape (506,)


# Using Normal Equation
Create an interactive session so that we don't have to load it any time

In [5]:
sess = tf.InteractiveSession()

Start creating variables within the session:

In [6]:
X = tf.constant(X_bias, name = "X")
y = tf.constant(yy, name = "y")

cov = tf.matmul(tf.transpose(X), X)
cov_inv = tf.matrix_inverse(cov)

xty = tf.matmul(X, y, transpose_a=True)
theta = tf.matmul(cov_inv, xty, name="theta")

Let's check to see if our variables are created in the correct session

In [7]:
theta.graph is sess.graph

True

To run the code, we need first to initialize all variables first:

In [8]:
init = tf.global_variables_initializer()
init.run()

Then we can calculate the variables:

In [9]:
theta_val = sess.run(theta)
theta_val

array([[  2.25328063e+01],
       [ -9.20411128e-01],
       [  1.08098058e+00],
       [  1.42967124e-01],
       [  6.82203460e-01],
       [ -2.06009246e+00],
       [  2.67064141e+00],
       [  2.11206306e-02],
       [ -3.10444805e+00],
       [  2.65878654e+00],
       [ -2.07589814e+00],
       [ -2.06215593e+00],
       [  8.56640436e-01],
       [ -3.74867982e+00]])

Let's see what will happen if we close the session.

In [10]:
sess.close()
sess.run(theta)

RuntimeError: Attempted to use a closed Session.

# Using Gradient Descent
We need to update the values of theta as we proceed down the gradient

In [ ]:
sess = tf.InteractiveSession()

theta = tf.random_normal((num_feat, 1))

# Some basic codes

In [12]:
sess = tf.InteractiveSession()
a = tf.zeros((2,2))
b = tf.ones((1,2))

print(tf.reduce_sum(b, reduction_indices=1).eval())

print(a.get_shape())

sess.close()

[ 2.]
(2, 2)


Simple Tensor flow codes to illustrate that we can initializer 1 variable.

In [63]:
x = tf.Variable(2.0)
y = 2.0 * (x ** 3)
z = 3.0 + y ** 2
grad_z = tf.gradients(z, [x, y])
with tf.Session() as sess:
    sess.run(x.initializer)
    print(sess.run(grad_z))

[768.0, 32.0]


In [62]:
### Changing the value of a variable at different nodes
W = tf.Variable(10)
assign_op = W.assign(100)

with tf.Session() as sess:
    sess.run(W.initializer)
    print(W.eval()) # >> 10
    print(sess.run(assign_op)) # >> 100

10
100


In [ ]:
# Similarly
# Example 3: each session maintains its own copy of variables
# Source: https://github.com/chiphuyen/stanford-tensorflow-tutorials/blob/master/2017/examples/02_variables.py
W = tf.Variable(10)
sess1 = tf.Session()
sess2 = tf.Session()

# You have to initialize W at each session
sess1.run(W.initializer)
sess2.run(W.initializer)

print(sess1.run(W.assign_add(10))) # >> 20
print(sess2.run(W.assign_sub(2))) # >> 8

print(sess1.run(W.assign_add(100))) # >> 120
print(sess2.run(W.assign_sub(50))) # >> -42

sess1.close()
sess2.close()

## Linear Regression
Notice that after we run the optimization, we have to use sess.run again to get the final values.

In [72]:
## Linear regression
## X, y like above
X = tf.constant(X_bias, dtype = tf.float32, name = "X") 
y = tf.constant(yy, dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_normal(shape=(num_feat+1, 1)),
                    name = "theta")

#theta = tf.random_normal(shape=(num_feat+1, 1))
#theta = tf.get_variable("weights", (num_feat+1, 1),
#                       initializer=tf.random_normal_initializer())
y_predict = tf.matmul(X, theta)
error = y_predict - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
opt = tf.train.GradientDescentOptimizer(learning_rate = 0.2)
opt_operation = opt.minimize(mse)

result = None

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    print("Init theta: ", sess.run(theta))
    
    sess.run(opt_operation)
    
    # After we run the optimization
    result = sess.run([theta, mse])

print(result)

Init theta:  [[ 0.86462492]
 [ 1.36153066]
 [ 0.21057315]
 [ 0.33255777]
 [-0.18070459]
 [ 0.72207183]
 [ 0.05724321]
 [-0.36944428]
 [-1.24097431]
 [-0.43374175]
 [ 0.194819  ]
 [-0.16698825]
 [-0.78600383]
 [-0.48621923]]
[array([[ 9.5318985 ],
       [-0.84208143],
       [ 1.93440461],
       [-2.13890767],
       [ 0.49438566],
       [-1.62086225],
       [ 2.77928591],
       [-2.31837916],
       [ 0.45307636],
       [-2.48947477],
       [-2.21376753],
       [-2.21165419],
       [ 1.08107591],
       [-3.66053152]], dtype=float32), 323.22888]


## Logistic Regression
Illustrate how we can load existing data into our program and run a batch optimization